In [1]:
"""
Simple examples of convolution to do some basic filters
Also demonstrates the use of TensorFlow data readers.

We will use some popular filters for our image.
It seems to be working with grayscale images, but not with rgb images.
It's probably because I didn't choose the right kernels for rgb images.

kernels for rgb images have dimensions 3 x 3 x 3 x 3
kernels for grayscale images have dimensions 3 x 3 x 1 x 1
"""
import os
import numpy as np
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import sys
sys.path.append('..')

from matplotlib import gridspec as gridspec
from matplotlib import pyplot as plt
import tensorflow as tf

import kernels

In [2]:
def read_one_image(filename):
    ''' This method is to show how to read image from a file into a tensor.
    The output is a tensor object.
    '''
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_image(image_string)
    image = tf.cast(image_decoded, tf.float32) / 256.0
    return image

def convolve(image, kernels, rgb=True, strides=[1, 3, 3, 1], padding='SAME'):
    images = [image[0]]
    for i, kernel in enumerate(kernels):
        filtered_image = tf.nn.conv2d(image, 
                                      kernel, 
                                      strides=strides,
                                      padding=padding)[0]
        if i == 2:
            filtered_image = tf.minimum(tf.nn.relu(filtered_image), 255)
        images.append(filtered_image)
    return images

In [3]:
def show_images(images, rgb=True):
    n_images=len(images)

    fig = plt.figure()
    fig.set_size_inches(30, 8)    
    gs = gridspec.GridSpec(1, len(images),figure=fig)
    
    for i, image in enumerate(images):
        ax = fig.add_subplot(gs[0, i])
        if rgb:
            ax.imshow(image)
        else: 
            image = image.reshape(image.shape[0], image.shape[1])
            ax.imshow(image, cmap='gray')
        plt.axis('off')
    #fig.set_size_inches(np.array(fig.get_size_inches()) * n_images*5)

    plt.show()

In [4]:
def show_images_v2(images, rgb=True, cols=1):
    n_images=len(images)

    fig = plt.figure()
    
    for i in range(n_images):
        ax = fig.add_subplot(cols,np.ceil(n_images/float(cols)), i+1)
        if rgb:
            ax.imshow(images[i])
        else:
            ax.imshow(images[i], cmap='Greys_r')
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

In [5]:
!ls ../data/*.jpg

../data/green-cascading-waterfall.jpg  ../data/puppy424x640.jpg
../data/mossy_rocks_landscape.jpg      ../data/puppy483x720.jpg
../data/puppy1200x650.jpg


In [6]:
def main(rgb=False,version=2, fsiz=None):
    #rgb = False
    if rgb:
        kernels_list = [kernels.BLUR_FILTER_RGB, 
                        kernels.SHARPEN_FILTER_RGB, 
                        kernels.EDGE_FILTER_RGB,
                        kernels.TOP_SOBEL_RGB,
                        kernels.EMBOSS_FILTER_RGB]
    else:
        kernels_list = [kernels.BLUR_FILTER,
                        kernels.SHARPEN_FILTER,
                        kernels.EDGE_FILTER,
                        kernels.TOP_SOBEL,
                        kernels.EMBOSS_FILTER]

    kernels_list = kernels_list[1:]
    image = read_one_image('../data/green-cascading-waterfall.jpg')
    if not rgb:
        image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, 0) # make it into a batch of 1 element
    images = convolve(image, kernels_list, rgb)
    with tf.Session() as sess:
        images = sess.run(images) # convert images from tensors to float values
    if version==1:
        show_images(images,rgb=rgb)
    elif version==2:
        show_images_v2(images, rgb)

In [7]:
main(rgb=False,version=1)

InternalError: Blas GEMV launch failed:  m=3, n=2304000
	 [[Node: rgb_to_grayscale/Tensordot/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](rgb_to_grayscale/Tensordot/Reshape, rgb_to_grayscale/Tensordot/Reshape_1)]]
	 [[Node: Minimum/_75 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_183_Minimum", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'rgb_to_grayscale/Tensordot/MatMul', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/

In [ ]:
main(rgb=True,version=2)

In [ ]:
def convolve_one_image(imagepath, kernel, rgb=True, strides=[1,3,3,1],padding='SAME'):
    ''' This method is to show how to read image from a file into a tensor.
    The output is a tensor object.
    '''
    image_string = tf.read_file(imagepath)
    image_decoded = tf.image.decode_image(image_string)
    image = tf.cast(image_decoded, tf.float32) / 256.0

    filtered_image = tf.nn.conv2d(image,kernel, strides=strides,padding=padding)
    #filtered_image = tf.minimum(tf.nn.relu(filtered_image), 255)
    return filtered_image

def show_one_image(image, rgb=True, figsize=(10,10)):
    fig = plt.figure(figsize=figsize)
    if rgb:
        plt.imshow(image)
    else:
        plt.imshow(image, cmap='Greys_r')
    plt.show()

In [ ]:
a = np.zeros([3, 3, 3, 3])
a[1, 1, :, :] = 5
a[0, 1, :, :] = -1
a[1, 0, :, :] = -1
a[2, 1, :, :] = -1
a[1, 2, :, :] = -1
a

In [ ]:
SHARPEN_FILTER_RGB = tf.constant(a, dtype=tf.float32)

In [ ]:
pic = '../data/green-cascading-waterfall.jpg'

with tf.Session() as sess:
    sharpened = sess.run(convolve_one_image(pic, SHARPEN_FILTER_RGB))
sharpened

In [ ]:
with tf.Session() as sess:
    spnd = sess.run(sharpened)
print(spnd)

In [ ]:
show_one_image(sharpened)